In [20]:
from astropy.coordinates import (CartesianRepresentation,
                                 UnitSphericalRepresentation)
from astropy.coordinates.matrix_utilities import rotation_matrix
import numpy as np
import astropy.units as u
from scipy.integrate import quad

n_incs = 10
n_phases = 30
n_spots = 3

spot_contrast = 0.7
spot_radii = 0.2 * np.ones((n_spots, n_incs))
inc_stellar = (180*np.random.rand(n_spots,n_incs) - 90) * u.deg
spot_lons = 360*np.random.rand(n_spots,n_incs) * u.deg
spot_lats = (20*np.random.rand(n_spots,n_incs) + 70) * u.deg
phases = np.linspace(0, 2*np.pi, n_phases)


def limb_darkening(u_ld, r):
    u1, u2 = u_ld
    mu = np.sqrt(1 - r**2)
    return (1 - u1 * (1 - mu) - u2 * (1 - mu)**2) / (1 - u1/3 - u2/6) / np.pi


def limb_darkening_normed(u_ld, r):
    return limb_darkening(u_ld, r)/limb_darkening(u_ld, 0)


def total_flux(u_ld):
    return 2 * np.pi * quad(lambda r: r * limb_darkening_normed(u_ld, r),
                            0, 1)[0]

u_ld = [0.5, 0.1]
f0 = total_flux(u_ld)

usr = UnitSphericalRepresentation(spot_lons, spot_lats)
cartesian = usr.represent_as(CartesianRepresentation)
rotate = rotation_matrix(phases[:, np.newaxis, np.newaxis],
                         axis='z')
tilt = rotation_matrix(inc_stellar - 90*u.deg, axis='y')
rotated_spot_positions = cartesian.transform(rotate)
tilted_spot_positions = rotated_spot_positions.transform(tilt)

r = np.ma.masked_array(np.sqrt(tilted_spot_positions.y**2 +
                               tilted_spot_positions.z**2),
                       mask=tilted_spot_positions.x < 0)
ld = limb_darkening_normed(u_ld, r)

f_spots = (np.pi * spot_radii**2 * (1 - spot_contrast) * ld *
           np.sqrt(1 - r**2))

delta_f = (1 - np.sum(f_spots/f0, axis=1)).data
delta_f/delta_f.max(axis=0)

<Quantity [[ 1.        , 0.99997261, 1.        , 0.99996378, 0.9999122 ,
             0.99995873, 1.        , 1.        , 0.99991864, 1.        ],
           [ 0.9999977 , 0.99997308, 0.99999575, 0.99996561, 0.99991474,
             0.99996046, 0.99999025, 0.99999488, 0.99992191, 0.99999355],
           [ 0.99999541, 0.99997359, 0.99999149, 0.99996739, 0.9999173 ,
             0.99996216, 0.99998046, 0.99998981, 0.99992515, 0.99998715],
           [ 0.99999315, 0.99997414, 0.99998723, 0.99996914, 0.99991991,
             0.99996385, 0.99997066, 0.99998477, 0.99992835, 0.99998081],
           [ 0.99999091, 0.99997472, 0.99998296, 0.99997084, 0.99992255,
             0.99996551, 0.99996082, 0.99997978, 0.99993152, 0.99997453],
           [ 0.99998868, 0.99997533, 0.99997869, 0.9999725 , 0.99992522,
             0.99996716, 0.99995096, 0.99997483, 0.99993466, 0.99996831],
           [ 0.99998648, 0.99997597, 0.99997441, 0.99997412, 0.99992794,
             0.99996878, 0.99994107, 0.999969

In [22]:
cartesian

<CartesianRepresentation (x, y, z) [dimensionless]
    [[( 0.01148623,  -5.29086344e-02,  0.9985333 ),
      (-0.10229262,  -6.46224895e-04,  0.99475414),
      (-0.16326352,   1.81514742e-01,  0.9697409 ),
      ( 0.28915467,   1.48147523e-03,  0.95728125),
      ( 0.17977883,   8.40515152e-03,  0.98367115),
      ( 0.12620149,   1.25761940e-01,  0.98400057),
      (-0.04829742,  -2.77404929e-01,  0.95953836),
      ( 0.17610171,   1.98980622e-01,  0.96405129),
      (-0.28104056,   2.73096008e-04,  0.95969585),
      ( 0.06037132,   1.23409275e-01,  0.99051777)],
     [(-0.10842045,  -2.66053336e-01,  0.95784165),
      (-0.30621854,   3.01591839e-02,  0.95148339),
      ( 0.12913017,   1.44724589e-04,  0.99162764),
      (-0.00043808,  -1.96617624e-02,  0.99980659),
      (-0.11748992,   2.22677662e-01,  0.96778654),
      (-0.26278903,  -1.91490832e-01,  0.94566019),
      ( 0.00526223,  -7.27045573e-02,  0.99733964),
      (-0.23129467,   1.20500947e-01,  0.9653923 ),
      ( 0.01